<a href="https://colab.research.google.com/github/MykhailoFokin/MachineLearning/blob/master/Naive_Bayes_text_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [70]:
# Basic test of algorithm (clear nltk with defaults)
# Move further to the next block for full implementation
from collections import Counter
import itertools
import math
import nltk
import copy
# nltk.download("popular")

# Train data
data = [["Chinese Beijing Chinese","0"],
        ["Chinese Chinese Shanghai","0"], 
        ["Chinese Macao","0"],
        ["Tokyo Japan Chinese","1"]]

class_count = {}
class_count_all = float(len(data))
word_count_all = {}
class_words_count = {}
wc_by_class = {item[1]: {} for item in data}
class_probability_template = copy.deepcopy(wc_by_class)
class_words_count = {item[1]: float(0) for item in data}
for line in data:
  if line[1] in class_count:
    class_count[line[1]] += 1
  else :
    class_count[line[1]] = float(1)
  tokenized_word=nltk.word_tokenize(line[0])
  fdist = nltk.FreqDist(tokenized_word)
  #fdist = nltk.FreqDist(w.lower() for w in tokenized_word)
  for word in fdist:
    #words_all += 1   # increase words counter
    class_words_count[line[1]] += fdist[word]
    # add/update word to class
    if word in wc_by_class[line[1]]:
      wc_by_class[line[1]][word] += fdist[word]
    else:
      wc_by_class[line[1]][word] = float(fdist[word])
    # add/update word to summary dict
    if word in word_count_all :
      word_count_all[word] += fdist[word]
    else :
      word_count_all[word] = float(fdist[word])
  #tokenized_words.append((dict(fdist), {"class" : line[1]}))

def calculate_conditional_probability(wc,count_c,v):
  '''
  == Input ==

  wc      : certain word count in appropriate class\n
  count_c : overall word count in appropriate class\n
  v       : unique word count in all classes
  
  == Output ==  
  result : probability for certain word for certain class
  '''
  return (1 + wc)/(count_c + v)

# Test
new = ["Chinese Chinese Chinese Tokyo Japan"]

tokenized_word=nltk.word_tokenize(new[0])
fdist = nltk.FreqDist(tokenized_word)
#print(Counter(tok['KeyName'] for tok in fdist))
x = 0
for key in wc_by_class:
  #print("Key of main dictionary: " + key)
  class_probability = class_count[key]/class_count_all
  for word in fdist:
    p1 = wc_by_class[key][word] if word in wc_by_class[key] else float(0)
    p2 = class_words_count[key]
    p3 = len(word_count_all)
    class_probability *= pow(calculate_conditional_probability(p1,p2,p3),fdist[word])
  #print("Class", key, class_probability)
  class_probability_template[key] = class_probability

print(class_probability_template)
print("Log10 : {d}", {k: math.log10(v) for k, v in class_probability_template.items()})
print("Log : {d}", {k: math.log(v) for k, v in class_probability_template.items()})
print("Log2 : {d}", {k: math.log2(v) for k, v in class_probability_template.items()})
print("Log1p : {d}", {k: math.log1p(v) for k, v in class_probability_template.items()})
print("Exp : {d}", {k: math.exp(v) for k, v in class_probability_template.items()})
print("Expm1 : {d}", {k: math.expm1(v) for k, v in class_probability_template.items()})

TypeError: ignored

In [0]:
# NLTK preparation
# Run it to get basic library content for futher usage
import nltk
nltk.download("popular")

In [0]:
# Import block
import nltk
#from collections import Counter
import itertools
import math
import copy
import matplotlib.pyplot as plt
from stop_words import get_stop_words

# Define block
class NaiveBayes :

  def __init__ (self) :
    self.ClassCount = {}
    word_count_all = {}
    self.ElementsByClass = {}  #   class_words_count = {}
    #wc_by_class = {item[1]: {} for item in data}
    # from stop_words import get_stop_words 
    # stop_words = list(get_stop_words('en'))  # About 900 words
    # from nltk.corpus import stopwords
    # list(stopwords.words('english')) # About 150 stopwords
    # stop_words.extend(nltk_words)
    self.stopwords = set(nltk.corpus.stopwords.words("english")) # import stopwords and remove duplicates

  def change_stopwords_list(self, whatlist = 'Default') :
    '''
    Optionally we can change default Enlgish stop words from NLTK

    == Input ==

    whatlist : string parameter for certain stop words list\n
       Default : Use only NLTK list. About 150 stopwords
       Alternative : Use stop_words library. About 900 words
       Extended : Use both together
    '''
    if whatlist == 'Default' :
      self.stopwords = set(nltk.corpus.stopwords.words("english")) # import stopwords and remove duplicates
    elif whatlist == 'Alternative' :
      self.stopwords = set(get_stop_words('en')) # import stopwords and remove duplicates
    elif whatlist == 'Extended' :
      nltkstopwords = list(nltk.corpus.stopwords.words("english"))
      stop_words = list(get_stop_words('en'))
      stop_words.extend(nltk_words)
      self.stopwords = set(stop_words)

  def fit(self, input_data, use_stop_words, use_stemming, use_lowercase) :
    #self.ClassCount = 
    self.ClassCountAll = float(len(input_data))  # class_count_all = float(len(data))
    self.ElementsByClass = {item[1]: float(0) for item in data}
    self.WordsByClass = {item[1]: {} for item in data}
    self.ClassProbabilityResult = copy.deepcopy(self.WordsByClass)
    self.UseStopWords = use_stop_words
    self.UseStemming = use_stemming
    self.UseLowercase = use_lowercase

    for line in input_data:
      if line[1] in self.ClassCount:
        self.ClassCount[line[1]] += 1
      else :
        self.ClassCount[line[1]] = float(1)
      tokenized_word = nltk.word_tokenize(line[0])
      #filtered_words = [word for word in tokenized_word if word not in stopwords.words('english')]
      fdist = nltk.FreqDist(w.lower() for w in tokenized_word) # stopwords in nltk in lowercase
      #fdist = nltk.FreqDist(w.lower() for w in filtered_words) # stopwords in nltk in lowercase
      filtered_words = [word for word in fdist if word not in stopwords.words('english')]
    for word in fdist:
      self.ElementsByClass[line[1]] += fdist[word]
      # add/update word to class
      if word in self.WordsByClass[line[1]]:
        self.WordsByClass[line[1]][word] += fdist[word]
      else:
        self.WordsByClass[line[1]][word] = float(fdist[word])
      # add/update word to summary dict
      if word in word_count_all :
        word_count_all[word] += fdist[word]
      else :
        word_count_all[word] = float(fdist[word])

tokenized_word=nltk.word_tokenize(new[0])
fdist = nltk.FreqDist(tokenized_word)
#print(Counter(tok['KeyName'] for tok in fdist))
x = 0
for key in self.WordsByClass:
  #print("Key of main dictionary: " + key)
  class_probability = self.ClassCount[key]/self.ClassCountAll
  for word in fdist:
    p1 = self.WordsByClass[key][word] if word in self.WordsByClass[key] else float(0)
    p2 = self.ElementsByClass[key]
    p3 = len(word_count_all)
    class_probability *= pow(calculate_conditional_probability(p1,p2,p3),fdist[word])
  #print("Class", key, class_probability)
  self.ClassProbabilityResult[key] = class_probability

  def print_freq_distrinbution(self, fdist):
    # Frequency Distribution Plot
    #FdistPrint = fdist.most_common(20) # take 20 most common words
    fdist.plot(30,cumulative=False)
    plt.show()

  def removing_stop_words(self, tokenized_string) :
     '''
    Removing stop words from base

    == Input ==

    fdsit : dict of tokens (dictionary of words from splitted string)
    
    == Output ==  

    stemmed_tokens : dict of preprocessed words
    '''   
    processed_word_list = []
    for word in tokenized_string:
        word = word.lower() # in case they arenet all lower cased
        if word not in self.stopwords.words("english"):
            processed_word_list.append(word)
    return processed_word_list
    #return [word for word in word_list if word not in self.stopwords.words('english')]
  
  def stemming(self, tokenized_string) :
    '''
    Stemming\n
    Stemming is a process of linguistic normalization, which reduces words
    to their word root word or chops off the derivational affixes.
    For example, connection, connected, connecting word reduce to a common word "connect".

    == Input ==

    fdsit : dict of tokens (dictionary of words from splitted string)
    
    == Output ==  

    stemmed_tokens : dict of preprocessed words
    '''

    ps = nltk.stem.PorterStemmer()

    stemmed_words=[]
    for w in tokenized_string:
        stemmed_words.append(ps.stem(w))

    return stemmed_words

   
